In [3]:
import numpy as np
import pandas as pd
from pymongo import MongoClient

In [5]:
# %load encodings.py
#!/bin/usr/env python

def get_encoding(file):
    import codecs
    encodings = ['UTF_8', 'ASCII','latin_1','iso8859_1','iso8859_15','cp1252','gb2312','euc_kr','cp1251','koi8_r','cp1250','iso8859_2',
 'cp1256','iso8859_6','big5','big5hkscs','cp037','cp424','cp437','cp500','cp737','cp775','cp850',
 'cp852','cp855','cp856','cp857','cp860','cp861','cp862','cp863','cp864','cp865','cp866','cp869',
 'cp874','cp875','cp932','cp949','cp950','cp1006','cp1026','cp1140','cp1250','cp1251','cp1252','cp1253','cp1254','cp1255',
 'cp1256','cp1257','cp1258','euc_jp','euc_jis_2004','euc_jisx0213','euc_kr','gb2312','gbk','gb18030','hz',
'iso2022_jp','iso2022_jp_1','iso2022_jp_2','iso2022_jp_2004','iso2022_jp_3','iso2022_jp_ext','iso2022_kr',
 'iso8859_2','iso8859_3','iso8859_4','iso8859_5','iso8859_6','iso8859_7','iso8859_8','iso8859_9','iso8859_10','iso8859_13','iso8859_14','iso8859_15',
 'johab','koi8_r','koi8_u','mac_cyrillic','mac_greek','mac_iceland','mac_latin2','mac_roman','mac_turkish',
 'shift_jis','shift_jis_2004','shift_jisx0213','utf_16','utf_16_be','utf_16_le','utf_7']
    
    for e in encodings:
        try:
            fh = codecs.open(file, 'r', encoding=e)
            fh.readlines()
            fh.seek(0)
        except UnicodeDecodeError:
            pass
        else:
            return e
            break

In [14]:
def load_transform(file):
    #loads and cleans the time as a time element, and drops the 'flag' column
    
    #loading params
    e = get_encoding(file_o)
    headers = ['target', 'id','date','drop_this','user','text']
    
    #create initial df
    df = pd.read_csv(file_o, names=headers, encoding=e)
    
    #clean time
    df.date = pd.to_datetime(df.date, utc=False)
    
    #drop column
    df = df.drop(['drop_this'], axis=1)
    
    return df

In [114]:
#functions
def conn_to_mongo(host, db, coll, statement="Connected"):
    
    try:
        Host =  MongoClient(host)
        DB = myclient[db]
        Collection = DB[coll]
        print(statement)
        return Host, DB, Collection
    
    except:
        print('Connection refused. Is your MongoDB running?')
        
    return Host, DB, Collection
        
def upload_df_to_mongodb(df, host, db, coll,orient='records'):
    # Connect to MongoDB
    Host, DB, Collection = conn_to_mongo(host, db, coll,statement="Connected, preparing to upload...")

    #prep df for transfer
    df.reset_index(inplace=True)
    data_dict = df.to_dict(orient)

    # Insert collection
    try: 
        Collection.insert_many(data_dict)
        print('Successfully uploaded the data.')
    except:
        print("Something went wrong; the data was not uploaded.")
    

In [ ]:
#params
file = 'data/training.1600000.processed.noemoticon.csv'
db = 'tweetydfdb'
coll = 'tweetydf'
client = "mongodb://localhost:27017/"

In [16]:
df = load_transform(file)

/Users/albertmorrow/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [ ]:
upload_df_to_mongodb(df, host, db, coll)